Adriver Case Study

Using your knowledge, the two datasets provided and your research, try to answer the following case.
You work in the Data Division of Adriver. The Marketing team needs to know if the VTC advertising campaign sponsored by SILVER Dream, a famous brand of senior accessories, worked well.
This was a test ad campaign with only 10 VTCs over two weeks. 
The brand wanted to reach the +65 years old, its target customer.
You have : 
- The tracking data of the VTCs on which the ad in question was displayed, anonymized. (vtc_tracking.csv)
- GPS position data from smartphones, also completely anonymized. (uuid_tracking.csv)
- Open Data from INSEE on Parisian demography (see link on Iris).
The goal, using this data, is to come up with an estimate of the campaign's KPIs. The marketing team asks you for the following figures:
- The size of the target audience in the study area.
- The number of Opportunities To See (OTV) generated by the campaign
- The average repetition of the campaign
- The audience of the campaign
- The advertising coverage
Feel free to take a look at this site, to understand the above terms.
You need to give the most accurate estimate of these KPIs, as well as a quick explanation of the methodology used. This will allow the sales team to give a coherent speech in front of their client and to justify the figures of the campaign.
At the next meeting, present your results in the form of a short PPT, a Word document or with your code, as you see fit.
Good luck!


## Import Libraries 

In [1]:
import pandas as pd
import numpy as np

## Load Datasets 

In [84]:
uuid = pd.read_csv("uuid_tracking.csv", sep = ";")
vehicleid = pd.read_csv("vehicle_tracking.csv")
insee = pd.read_csv("base-ic-evol-struct-pop-2017.CSV", sep = ";")
inseemeta = pd.read_csv("meta_base-ic-evol-struct-pop-2017.CSV", sep = ";")
com = pd.read_csv("commune2021.csv")

C:\Users\zhrht\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [85]:
uuid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80650 entries, 0 to 80649
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pseudo_id         80650 non-null  object 
 1   consentMedia      80650 non-null  bool   
 2   consentData       80650 non-null  bool   
 3   date_uuid         80650 non-null  object 
 4   speed             80648 non-null  float64
 5   accuracy          80650 non-null  int64  
 6   altitude          78889 non-null  float64
 7   verticalAccuracy  78889 non-null  float64
 8   os                80650 non-null  object 
 9   deviceModel       80650 non-null  object 
 10  location          80650 non-null  object 
dtypes: bool(2), float64(3), int64(1), object(5)
memory usage: 5.7+ MB


In [79]:
uuid.columns

Index(['pseudo_id', 'consentMedia', 'consentData', 'date_uuid', 'speed',
       'accuracy', 'altitude', 'verticalAccuracy', 'os', 'deviceModel',
       'location', 'date_uuid_new'],
      dtype='object')

In [80]:
uuid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80650 entries, 0 to 80649
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pseudo_id         80650 non-null  object 
 1   consentMedia      80650 non-null  bool   
 2   consentData       80650 non-null  bool   
 3   date_uuid         80650 non-null  object 
 4   speed             80648 non-null  float64
 5   accuracy          80650 non-null  int64  
 6   altitude          78889 non-null  float64
 7   verticalAccuracy  78889 non-null  float64
 8   os                80650 non-null  object 
 9   deviceModel       80650 non-null  object 
 10  location          80650 non-null  object 
 11  date_uuid_new     80650 non-null  object 
dtypes: bool(2), float64(3), int64(1), object(6)
memory usage: 6.3+ MB


In [81]:
vehicleid.tail()

,vehicle_id,trip_id,date_vtc,location
41995,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-03 21:08:49,POINT(2.3235375123 48.869646472)
41996,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:06:17,POINT(2.3218290086 48.8660899855)
41997,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:52:34,POINT(2.31495174035 48.855575531)
41998,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 01:01:30,POINT(2.3225859406 48.8734018485)
41999,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 01:40:07,POINT(2.3468510191 48.883030823)


In [15]:
vehicleid['date_vtc'] =  pd.to_datetime(vehicleid['date_vtc'])

In [18]:
vehicleid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   vehicle_id  42000 non-null  int64         
 1   trip_id     42000 non-null  object        
 2   date_vtc    42000 non-null  datetime64[ns]
 3   location    42000 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.3+ MB


In [20]:
vehicleid.sort_values(by='date_vtc', ascending=False)

,vehicle_id,trip_id,date_vtc,location
137,654,e2dc400a-f494-4f9b-9a42-acbd4fe7630e,2021-09-14 23:59:44,POINT(2.3169846435 48.875122968)
6439,81795,f8791a9c-730a-437b-9f60-0272d3974a06,2021-09-14 23:59:43,POINT(2.34094694915 48.875151416)
3817,654,e2dc400a-f494-4f9b-9a42-acbd4fe7630e,2021-09-14 23:59:40,POINT(2.316725124 48.875151416)
12243,81795,f8791a9c-730a-437b-9f60-0272d3974a06,2021-09-14 23:59:38,POINT(2.34015397276667 48.8748859026667)
17906,81795,f8791a9c-730a-437b-9f60-0272d3974a06,2021-09-14 23:59:35,POINT(2.3395628449 48.874724698)
...,...,...,...,...
18245,89665,7c890f77-6002-4925-9ca4-1905dd70954d,2021-09-01 00:01:52,POINT(2.3020406424 48.864340101)
995,89665,7c890f77-6002-4925-9ca4-1905dd70954d,2021-09-01 00:01:14,POINT(2.3014999767 48.86471)
6774,89665,7c890f77-6002-4925-9ca4-1905dd70954d,2021-09-01 00:00:57,POINT(2.30171624295 48.8647526805)
18254,89665,7c890f77-6002-4925-9ca4-1905dd70954d,2021-09-01 00:00:28,POINT(2.3019325093 48.864766908)


In [86]:
uuid['date_uuid'] =  pd.to_datetime(uuid['date_uuid'])
uuid.sort_values(by='date_uuid', ascending=False)

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location
7524,1A941,True,True,2021-12-09 23:59:00,167.579052,65,33.435486,29.832605,ios,"iPad11,1",POINT(2.32003057003021 48.8623678088188)
65586,8601b,True,True,2021-12-09 23:49:00,0.000000,13,81.000000,3.000000,and,samsung SM-A415F,POINT(2.31854194402695 48.8574475049973)
14200,0,True,True,2021-12-09 23:49:00,18.571991,165,37.529587,38.074486,ios,"iPhone10,4",POINT(2.31442844867706 48.8600109815598)
35349,60303,True,True,2021-12-09 23:48:00,0.000000,4,NaN,NaN,ios,"iPhone10,5",POINT(2.32138758897781 48.8803889751434)
32779,4a158,True,True,2021-12-09 23:48:00,4.434877,15,100.000000,3.000000,and,samsung SM-G965F,POINT(2.33505201339722 48.8805416226387)
...,...,...,...,...,...,...,...,...,...,...,...
61077,0d08a,True,True,2021-01-09 01:55:00,0.000000,23,92.000000,5.000000,and,samsung SM-G998B,POINT(2.32336157560348 48.8800246119499)
74383,4d229,True,True,2021-01-09 01:54:00,0.000000,18,79.000000,3.000000,and,samsung SM-A515F,POINT(2.34487599134445 48.8740048408508)
33863,4d229,True,True,2021-01-09 01:39:00,0.000000,12,NaN,NaN,and,samsung SM-A515F,POINT(2.34320288896561 48.8722257018089)
25760,3981E,True,True,2021-01-09 00:08:00,15.666392,165,49.272331,29.291504,ios,"iPhone11,2",POINT(2.30103427171707 48.8716050982475)


In [24]:
uuid

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location
0,0eb25,True,True,2021-09-14 13:58:00,0.000000,15,169.000000,0.000000,and,Xiaomi MI MAX 2,POINT(2.31323444843292 48.8488971590996)
1,C3473,True,True,2021-09-09 18:27:00,0.000000,10,30.974026,4.000000,ios,"iPhone8,1",POINT(2.31348294019699 48.8712033033371)
2,49cef,True,True,2021-09-14 12:50:00,13.046592,21,103.000000,3.000000,and,motorola Moto G (5),POINT(2.31104499101639 48.8806031346321)
3,93e8c,True,False,2021-09-14 18:21:00,10.886680,2,71.000000,108.000000,and,samsung SM-A530F,POINT(2.31671500205994 48.8471108078957)
4,56D83,True,True,2021-11-09 16:05:00,3.578376,10,59.289700,4.000000,ios,"iPhone7,2",POINT(2.32746052742004 48.8833501338959)
...,...,...,...,...,...,...,...,...,...,...,...
80645,73009,True,True,2021-08-09 12:39:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32523882389069 48.858107984066)
80646,73009,True,True,2021-08-09 11:52:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524847984314 48.8581141829491)
80647,73009,True,True,2021-08-09 13:14:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.325235247612 48.8581080436707)
80648,73009,True,True,2021-08-09 11:44:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524710893631 48.8581057190895)


In [31]:
vehicleid['trip_id'].nunique()

303

In [30]:
uuid['pseudo_id'].nunique()

6373

In [25]:
vehicleid['vehicle_id'].nunique()

10

#### From initial analysis we found 10 VTCs had made 303 trips from 2021-09-14 23:59:44 to 2021-09-01 00:00:21
#### 6373 devices are recorded

In [43]:
vehicleid['location'].value_counts()[:10]

POINT(2.3312365925 48.8732027065)    75
POINT(2.3357133048 48.8725768275)    74
POINT(2.3228238335 48.8677829045)    74
POINT(2.3236888987 48.8689920975)    69
POINT(2.3458345675 48.871012096)     69
POINT(2.3396493514 48.8720362895)    63
POINT(2.335410532 48.872605277)      62
POINT(2.31672512395 48.864894949)    62
POINT(2.3340264277 48.872804421)     61
POINT(2.33906543245 48.872121638)    60
Name: location, dtype: int64

In [35]:
insee.head(2)

,IRIS,COM,TYP_IRIS,MODIF_IRIS,LAB_IRIS,P17_POP,P17_POP0002,P17_POP0305,P17_POP0610,P17_POP1117,...,C17_F15P_CS4,C17_F15P_CS5,C17_F15P_CS6,C17_F15P_CS7,C17_F15P_CS8,P17_POP_FR,P17_POP_ETR,P17_POP_IMM,P17_PMEN,P17_PHORMEN
0,10010000,1001,Z,0,5,776.0,29.145423,18.057932,57.197304,90.335473,...,50.819279,85.780891,30.416743,71.592413,15.189774,767.876506,8.123494,20.282806,776.0,0.0
1,10020000,1002,Z,0,5,248.0,14.288066,10.205761,14.288066,24.493827,...,20.411523,25.514403,0.000000,20.411523,0.000000,245.958848,2.041152,3.061728,248.0,0.0


In [26]:
insee.columns

Index(['IRIS', 'COM', 'TYP_IRIS', 'MODIF_IRIS', 'LAB_IRIS', 'P17_POP',
       'P17_POP0002', 'P17_POP0305', 'P17_POP0610', 'P17_POP1117',
       'P17_POP1824', 'P17_POP2539', 'P17_POP4054', 'P17_POP5564',
       'P17_POP6579', 'P17_POP80P', 'P17_POP0014', 'P17_POP1529',
       'P17_POP3044', 'P17_POP4559', 'P17_POP6074', 'P17_POP75P',
       'P17_POP0019', 'P17_POP2064', 'P17_POP65P', 'P17_POPH', 'P17_H0014',
       'P17_H1529', 'P17_H3044', 'P17_H4559', 'P17_H6074', 'P17_H75P',
       'P17_H0019', 'P17_H2064', 'P17_H65P', 'P17_POPF', 'P17_F0014',
       'P17_F1529', 'P17_F3044', 'P17_F4559', 'P17_F6074', 'P17_F75P',
       'P17_F0019', 'P17_F2064', 'P17_F65P', 'C17_POP15P', 'C17_POP15P_CS1',
       'C17_POP15P_CS2', 'C17_POP15P_CS3', 'C17_POP15P_CS4', 'C17_POP15P_CS5',
       'C17_POP15P_CS6', 'C17_POP15P_CS7', 'C17_POP15P_CS8', 'C17_H15P',
       'C17_H15P_CS1', 'C17_H15P_CS2', 'C17_H15P_CS3', 'C17_H15P_CS4',
       'C17_H15P_CS5', 'C17_H15P_CS6', 'C17_H15P_CS7', 'C17_H15P_CS8',
    

In [27]:
inseemeta.columns

Index(['COD_VAR', 'LIB_VAR', 'LIB_VAR_LONG', 'COD_MOD', 'LIB_MOD', 'TYPE_VAR',
       'LONG_VAR'],
      dtype='object')

In [28]:
inseemeta

,COD_VAR,LIB_VAR,LIB_VAR_LONG,COD_MOD,LIB_MOD,TYPE_VAR,LONG_VAR
0,P17_POP,Population en 2017 (princ),Population,NaN,NaN,NUM,16
1,P17_POP0002,Pop 0-2 ans en 2017 (princ),Nombre de personnes de 0 à 2 ans,NaN,NaN,NUM,17
2,P17_POP0305,Pop 3-5 ans en 2017 (princ),Nombre de personnes de 3 à 5 ans,NaN,NaN,NUM,17
3,P17_POP0610,Pop 6-10 ans en 2017 (princ),Nombre de personnes de 6 à 10 ans,NaN,NaN,NUM,17
4,P17_POP1117,Pop 11-17 ans en 2017 (princ),Nombre de personnes de 11 à 17 ans,NaN,NaN,NUM,17
...,...,...,...,...,...,...,...
84356,LAB_IRIS,Label de l'IRIS,Label de qualité de l'IRIS,2,Commune irisée de 10000 hab ou plus. Utiliser ...,CHAR,1
84357,LAB_IRIS,Label de l'IRIS,Label de qualité de l'IRIS,3,Commune irisée de 10000 hab ou plus. IRIS atyp...,CHAR,1
84358,LAB_IRIS,Label de l'IRIS,Label de qualité de l'IRIS,4,Commune irisée de moins de 10000 hab.,CHAR,1
84359,LAB_IRIS,Label de l'IRIS,Label de qualité de l'IRIS,5,Commune non irisée de moins de 10000 hab.,CHAR,1


#### from insee data we extract the target audience data 

In [39]:
dfpop = insee[['IRIS', 'COM', 'TYP_IRIS', 'MODIF_IRIS', 'LAB_IRIS', 'P17_POP65P']]

In [40]:
dfpop['P17_POP65P'].sum()

12918050.761881253

### We use INSEE GEO CODES TO GET COMMUNE CODES  

In [38]:
com

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37737,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37738,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37739,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37740,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


In [44]:
com.loc[com['DEP'] == '75']

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
31658,COM,75056,11.0,75,75C,751,0,PARIS,Paris,Paris,NaN,NaN
31659,ARM,75101,11.0,75,75C,751,0,PARIS 1ER ARRONDISSEMENT,Paris 1er Arrondissement,Paris 1er Arrondissement,NaN,75056.0
31660,ARM,75102,11.0,75,75C,751,0,PARIS 2E ARRONDISSEMENT,Paris 2e Arrondissement,Paris 2e Arrondissement,NaN,75056.0
31661,ARM,75103,11.0,75,75C,751,0,PARIS 3E ARRONDISSEMENT,Paris 3e Arrondissement,Paris 3e Arrondissement,NaN,75056.0
31662,ARM,75104,11.0,75,75C,751,0,PARIS 4E ARRONDISSEMENT,Paris 4e Arrondissement,Paris 4e Arrondissement,NaN,75056.0
31663,ARM,75105,11.0,75,75C,751,0,PARIS 5E ARRONDISSEMENT,Paris 5e Arrondissement,Paris 5e Arrondissement,NaN,75056.0
31664,ARM,75106,11.0,75,75C,751,0,PARIS 6E ARRONDISSEMENT,Paris 6e Arrondissement,Paris 6e Arrondissement,NaN,75056.0
31665,ARM,75107,11.0,75,75C,751,0,PARIS 7E ARRONDISSEMENT,Paris 7e Arrondissement,Paris 7e Arrondissement,NaN,75056.0
31666,ARM,75108,11.0,75,75C,751,0,PARIS 8E ARRONDISSEMENT,Paris 8e Arrondissement,Paris 8e Arrondissement,NaN,75056.0
31667,ARM,75109,11.0,75,75C,751,0,PARIS 9E ARRONDISSEMENT,Paris 9e Arrondissement,Paris 9e Arrondissement,NaN,75056.0


### Based on Vechile location data, we found out that most of its trips were in 48.87320 2.3312365 latitutes and langitutde which falls between  ARRONDISSEMENTS mostly surrounding 1ST.

#### Hypothesis
#### we select only parts were VTCs travelled and the population of target audience
#### we narrowed it to 7 ARRONDISSEMENTS, 7th, 6th, 1th, 2th, 3th, 8th, 10th

In [55]:
communecodes = [75107, 75106, 75108, 75101, 75102, 75103, 75110]

In [57]:
dftarget = dfpop.loc[dfpop['COM'].isin(communecodes)]

In [59]:
dftarget['P17_POP65P'].sum()

47700.57127948023

# KPI 1
## - The size of the target audience in the study area.
### 47700 Target Audience over 65 SPECIFIC THE SELECTED ARRONDISSEMENTS (Based on VTCs trip LOCATIONS)
### 12918050 Target Audience over 65 (FRANCE)


## KPI 2
### The number of Opportunities To See (OTV) generated by the campaign


In [63]:
vehicleid

,vehicle_id,trip_id,date_vtc,location
0,654,d2d6bfd5-83ae-4bd6-bb9d-d785316b0fec,2021-09-07 02:16:33,POINT(2.31166449265 48.873145809)
1,654,c3ab20fd-1e88-4a33-9017-80dcb661f3ed,2021-09-06 22:51:37,POINT(2.3432826252 48.8793899515)
2,654,e93b5c42-db55-4f17-817d-77b0951facf1,2021-09-14 18:01:34,POINT(2.32059629065 48.8656062835)
3,654,9a05e361-0930-4811-aca7-379628d75c1b,2021-09-03 02:02:34,POINT(2.30513325045 48.874724698)
4,654,21f577a1-11b4-489c-b31b-ff0fa211320f,2021-09-06 17:50:14,POINT(2.3072959134 48.874127286)
...,...,...,...,...
41995,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-03 21:08:49,POINT(2.3235375123 48.869646472)
41996,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:06:17,POINT(2.3218290086 48.8660899855)
41997,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:52:34,POINT(2.31495174035 48.855575531)
41998,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 01:01:30,POINT(2.3225859406 48.8734018485)


In [64]:
uuid

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location
0,0eb25,True,True,14/09/2021 13:58,0.000000,15,169.000000,0.000000,and,Xiaomi MI MAX 2,POINT(2.31323444843292 48.8488971590996)
1,C3473,True,True,09/09/2021 18:27,0.000000,10,30.974026,4.000000,ios,"iPhone8,1",POINT(2.31348294019699 48.8712033033371)
2,49cef,True,True,14/09/2021 12:50,13.046592,21,103.000000,3.000000,and,motorola Moto G (5),POINT(2.31104499101639 48.8806031346321)
3,93e8c,True,False,14/09/2021 18:21,10.886680,2,71.000000,108.000000,and,samsung SM-A530F,POINT(2.31671500205994 48.8471108078957)
4,56D83,True,True,11/09/2021 16:05,3.578376,10,59.289700,4.000000,ios,"iPhone7,2",POINT(2.32746052742004 48.8833501338959)
...,...,...,...,...,...,...,...,...,...,...,...
80645,73009,True,True,08/09/2021 12:39,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32523882389069 48.858107984066)
80646,73009,True,True,08/09/2021 11:52,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524847984314 48.8581141829491)
80647,73009,True,True,08/09/2021 13:14,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.325235247612 48.8581080436707)
80648,73009,True,True,08/09/2021 11:44,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524710893631 48.8581057190895)


#### Differnt format of dates found. lets chnage both datsets date into a new format for analysis 

In [88]:
uuid['date_uuid_new'] = uuid['date_uuid'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%B-%d %I:%M %p'))

In [89]:
uuid

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location,date_uuid_new
0,0eb25,True,True,2021-09-14 13:58:00,0.000000,15,169.000000,0.000000,and,Xiaomi MI MAX 2,POINT(2.31323444843292 48.8488971590996),2021-September-14 01:58 PM
1,C3473,True,True,2021-09-09 18:27:00,0.000000,10,30.974026,4.000000,ios,"iPhone8,1",POINT(2.31348294019699 48.8712033033371),2021-September-09 06:27 PM
2,49cef,True,True,2021-09-14 12:50:00,13.046592,21,103.000000,3.000000,and,motorola Moto G (5),POINT(2.31104499101639 48.8806031346321),2021-September-14 12:50 PM
3,93e8c,True,False,2021-09-14 18:21:00,10.886680,2,71.000000,108.000000,and,samsung SM-A530F,POINT(2.31671500205994 48.8471108078957),2021-September-14 06:21 PM
4,56D83,True,True,2021-11-09 16:05:00,3.578376,10,59.289700,4.000000,ios,"iPhone7,2",POINT(2.32746052742004 48.8833501338959),2021-November-09 04:05 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
80645,73009,True,True,2021-08-09 12:39:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32523882389069 48.858107984066),2021-August-09 12:39 PM
80646,73009,True,True,2021-08-09 11:52:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524847984314 48.8581141829491),2021-August-09 11:52 AM
80647,73009,True,True,2021-08-09 13:14:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.325235247612 48.8581080436707),2021-August-09 01:14 PM
80648,73009,True,True,2021-08-09 11:44:00,0.000000,2000,50.126553,60.827324,ios,"iPhone10,4",POINT(2.32524710893631 48.8581057190895),2021-August-09 11:44 AM


In [90]:
vehicleid

,vehicle_id,trip_id,date_vtc,location
0,654,d2d6bfd5-83ae-4bd6-bb9d-d785316b0fec,2021-09-07 02:16:33,POINT(2.31166449265 48.873145809)
1,654,c3ab20fd-1e88-4a33-9017-80dcb661f3ed,2021-09-06 22:51:37,POINT(2.3432826252 48.8793899515)
2,654,e93b5c42-db55-4f17-817d-77b0951facf1,2021-09-14 18:01:34,POINT(2.32059629065 48.8656062835)
3,654,9a05e361-0930-4811-aca7-379628d75c1b,2021-09-03 02:02:34,POINT(2.30513325045 48.874724698)
4,654,21f577a1-11b4-489c-b31b-ff0fa211320f,2021-09-06 17:50:14,POINT(2.3072959134 48.874127286)
...,...,...,...,...
41995,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-03 21:08:49,POINT(2.3235375123 48.869646472)
41996,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:06:17,POINT(2.3218290086 48.8660899855)
41997,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:52:34,POINT(2.31495174035 48.855575531)
41998,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 01:01:30,POINT(2.3225859406 48.8734018485)


In [91]:
vehicleid['date_vtc_new'] = vehicleid['date_vtc'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%B-%d %I:%M %p'))

In [92]:
vehicleid

,vehicle_id,trip_id,date_vtc,location,date_vtc_new
0,654,d2d6bfd5-83ae-4bd6-bb9d-d785316b0fec,2021-09-07 02:16:33,POINT(2.31166449265 48.873145809),2021-September-07 02:16 AM
1,654,c3ab20fd-1e88-4a33-9017-80dcb661f3ed,2021-09-06 22:51:37,POINT(2.3432826252 48.8793899515),2021-September-06 10:51 PM
2,654,e93b5c42-db55-4f17-817d-77b0951facf1,2021-09-14 18:01:34,POINT(2.32059629065 48.8656062835),2021-September-14 06:01 PM
3,654,9a05e361-0930-4811-aca7-379628d75c1b,2021-09-03 02:02:34,POINT(2.30513325045 48.874724698),2021-September-03 02:02 AM
4,654,21f577a1-11b4-489c-b31b-ff0fa211320f,2021-09-06 17:50:14,POINT(2.3072959134 48.874127286),2021-September-06 05:50 PM
...,...,...,...,...,...
41995,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-03 21:08:49,POINT(2.3235375123 48.869646472),2021-September-03 09:08 PM
41996,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:06:17,POINT(2.3218290086 48.8660899855),2021-September-04 12:06 AM
41997,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 00:52:34,POINT(2.31495174035 48.855575531),2021-September-04 12:52 AM
41998,59518,ff461cde-9f0e-49a4-9cbe-f53f6e5fa60b,2021-09-04 01:01:30,POINT(2.3225859406 48.8734018485),2021-September-04 01:01 AM


In [93]:
vehicleid.sort_values(by='date_vtc_new', ascending=False)

,vehicle_id,trip_id,date_vtc,location,date_vtc_new
24740,654,4d4d17a6-8065-4129-bd79-d67932ec0e74,2021-09-14 00:59:26,POINT(2.3085286313 48.875037625),2021-September-14 12:59 AM
24614,654,4d4d17a6-8065-4129-bd79-d67932ec0e74,2021-09-14 00:59:55,POINT(2.30571716945 48.874966505),2021-September-14 12:59 AM
24765,654,4d4d17a6-8065-4129-bd79-d67932ec0e74,2021-09-14 00:59:43,POINT(2.30722382463333 48.874980729),2021-September-14 12:59 AM
24701,654,4d4d17a6-8065-4129-bd79-d67932ec0e74,2021-09-14 00:59:08,POINT(2.312399798 48.8751940875),2021-September-14 12:59 AM
24677,654,4d4d17a6-8065-4129-bd79-d67932ec0e74,2021-09-14 00:59:30,POINT(2.30885303075 48.8750518485),2021-September-14 12:59 AM
...,...,...,...,...,...
29663,49099,883d3ac3-e877-4ac4-acd9-4f6f330b8a9f,2021-09-01 01:00:59,POINT(2.33681626295 48.872406132),2021-September-01 01:00 AM
29683,49099,883d3ac3-e877-4ac4-acd9-4f6f330b8a9f,2021-09-01 01:00:21,POINT(2.3388275395 48.873558315),2021-September-01 01:00 AM
8242,81779,2305c57b-ce59-494d-8ff8-c46681a4b47a,2021-09-01 01:00:08,POINT(2.30645247485 48.851235363),2021-September-01 01:00 AM
29598,49099,883d3ac3-e877-4ac4-acd9-4f6f330b8a9f,2021-09-01 01:00:16,POINT(2.33815711395 48.872249661),2021-September-01 01:00 AM


In [94]:
uuid.sort_values(by='date_uuid_new', ascending=False)

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location,date_uuid_new
11863,49cef,True,True,2021-09-14 12:59:00,10.127128,24,107.000000,3.000000,and,motorola Moto G (5),POINT(2.33360707759857 48.8833734989166),2021-September-14 12:59 PM
43465,2A993,True,True,2021-09-14 12:59:00,-1.000000,65,0.000000,0.000000,ios,"iPhone9,3",POINT(2.31876122951508 48.8756526112556),2021-September-14 12:59 PM
55236,596A0,True,True,2021-09-14 12:59:00,-1.000000,38,0.000000,0.000000,ios,"iPhone6,2",POINT(2.31066530942917 48.8712732791901),2021-September-14 12:59 PM
78138,3b6ae,True,True,2021-09-14 12:59:00,18.333261,73,91.000000,3.000000,and,Xiaomi Mi 9T,POINT(2.30270832777023 48.8770411610603),2021-September-14 12:59 PM
72353,f92ae,True,True,2021-09-14 12:59:00,2.660160,11,86.000000,3.000000,and,samsung SM-G950F,POINT(2.30290728807449 48.8691599965096),2021-September-14 12:59 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
9351,25D3B,True,True,2021-04-09 13:05:00,14.403890,17,39.041714,4.000000,ios,"iPhone13,4",POINT(2.30138051509857 48.8681710958481),2021-April-09 01:05 PM
20431,EB0A8,True,True,2021-04-09 01:05:00,6.948732,2190,50.765698,5.042618,ios,"iPhone13,4",POINT(2.3349152803421 48.8800823092461),2021-April-09 01:05 AM
31891,1A023,True,True,2021-04-09 01:04:00,1.310769,48,25.392495,3.000000,ios,"iPhone9,3",POINT(2.30999904870987 48.8691719770432),2021-April-09 01:04 AM
76340,af7ba,True,True,2021-04-09 13:03:00,7.317928,500,83.000000,3.000000,and,HUAWEI COL-L29,POINT(2.3189445734024 48.8770438432693),2021-April-09 01:03 PM


In [96]:
res = pd.merge(uuid,vehicleid,on='location')
res

,pseudo_id,consentMedia,consentData,date_uuid,speed,accuracy,altitude,verticalAccuracy,os,deviceModel,location,date_uuid_new,vehicle_id,trip_id,date_vtc,date_vtc_new


In [ ]:
https://en.ryte.com/wiki/Opportunity_To_See